## Importing packages

Importing all needed packages for the full notebook. Only needs to be ran once. 

In [2]:
import gzip as gz
import json
import pandas as pd
import spacy as sp
from gensim.models import Word2Vec


## Importing data

In [3]:
# Setting local file path
path = "release/train.jsonl.gz"

# Creating list entity to hold full set of loaded data
data = []

# Using gz to set path to zip file and iteritavly load each json line
with gz.open(path) as f:
    for ln in f:
        obj = json.loads(ln)
        data.append(obj)

In [4]:
# Extracting appropriate data and transforming to pandas dataframe
df = pd.DataFrame(data[0:100000])
df_extractive = df[df.density_bin == 'extractive']

### Sentence splitting

In [5]:
nlp = sp.load("en_core_web_sm", disable=["tagger", "parser", "ner", "textcat"])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [6]:
# SENTENCE SPLITTING FUNCTION
def sentence_splitting(doc):
    doc = nlp(doc['text'])
    sentences = [sent.string.strip() for sent in doc.sents]
    
    return sentences

In [8]:
sentences = sentence_splitting(df_extractive.iloc[0])